In [2]:
import pandas as pd
cohort_file= '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts/cohort/all_cohorts.csv'

df = pd.read_csv(cohort_file)

numbers = df.groupby(['study', 'grp']).count().person_id

means = (df
         .groupby(['grp'])
         .mean()
         .filter(['age', 'ascvd_10yr', 'censored_10yr', 'event_time_10yr'])
         .round(3)
         .reset_index()
        )

means_all = (df
             .assign(grp='all')
             .groupby(['grp'])
             .mean()
             .filter(['age', 'ascvd_10yr', 'censored_10yr', 'event_time_10yr'])
             .round(3)
             .reset_index()
            )


means = pd.concat([means, means_all]).reset_index()#.set_index(['grp']).sort_values(by='study')

counts = (df
          .groupby('grp')
          .count()
          .filter(['person_id'])
          .rename(columns={'person_id': 'n'})
          .reset_index()
          .append({'n': df.count().person_id, 'grp': 'all'}, ignore_index=True)
          .reset_index()
         )

(means.assign(counts=counts.n).filter(['grp', 'counts', 'age', 'ascvd_10yr', 'censored_10yr'])
 .rename(columns={'grp': 'group'})
 .assign(group = lambda x: x.group.map({1: 'Black women', 2: 'White women',
                                       3: 'Black men', 4: 'White men', 'all': 'all'}))
)

,group,counts,age,ascvd_10yr,censored_10yr
0,Black women,5354,55.506,0.056,0.100
1,White women,8765,57.146,0.060,0.062
2,Black men,3848,56.292,0.077,0.125
3,White men,7652,56.516,0.103,0.073
4,all,25619,56.487,0.075,0.083


In [8]:
import os
args = {'base_path': '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts'}
experiment = 'apr14_erm'
aggregate_path = os.path.join(args['base_path'], 'experiments', 
                                      experiment, 'performance',
                                      'all')
preds_path = os.path.join(aggregate_path, 'predictions.csv')
preds = pd.read_csv(preds_path)

In [42]:
preds

,phase,outputs,pred_probs,labels,person_id,weights,group,treat,ldlc,relative_risk,fold_id,config_id,model_id,model_type
0,val,-1.345998,0.018933,0,1,1.069048,2,0,141.679,1.000000,1,0,0,erm
1,val,-1.222153,0.009032,0,12,1.069048,2,0,140.901,1.000000,1,0,0,erm
2,val,-1.245554,0.011558,0,13,1.069048,2,0,133.879,1.000000,1,0,0,erm
3,val,-1.244832,0.027471,0,20,1.069048,2,0,108.800,1.000000,1,0,0,erm
4,val,-1.882649,0.048856,0,44,1.069048,2,0,133.294,1.000000,1,0,0,erm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65469,eval,-1.711152,0.081717,1,23450,1.132407,3,1,167.000,0.689061,10,0,0,erm
65470,eval,-1.691981,0.301015,1,23472,1.122530,3,2,144.000,0.585539,10,0,0,erm
65471,eval,-0.864329,0.064934,1,23476,1.126947,3,0,79.000,1.000000,10,0,0,erm
65472,eval,-2.259879,0.249096,1,23487,1.034370,3,2,111.000,0.661949,10,0,0,erm


In [108]:
(preds
 .filter(['phase', 'person_id', 'labels', 'weights', 'group'])
 .groupby(['group', 'labels'])
 .agg({'person_id': 'count', 'weights': 'mean'})
 .rename(columns={'person_id': 'n', 'weights': 'mean_ipcw'})
 .assign(weighted_n = lambda x: x.n*x.mean_ipcw)
 .filter(['n'])
 .reset_index()
 .assign(labels = lambda x: x.labels.astype(str))
 .pivot(index='group', columns='labels', values='n')
 .assign(ascvd_10yr_weighted = lambda x: x['1']/(x['0']+x['1']))
 #.filter('ascvd_10yr_weighted')
 .reset_index()
 .rename_axis('', axis='index')
 .filter(['group','ascvd_10yr_weighted'])
)

labels,group,ascvd_10yr_weighted
,,
0,1,0.060911
1,2,0.061574
2,3,0.087601
3,4,0.108217


In [74]:
(preds
 .filter(['phase', 'person_id', 'labels', 'weights', 'group'])
 .groupby(['group', 'labels'])
 .agg({'person_id': 'count', 'weights': 'mean'})
 .rename(columns={'person_id': 'n', 'weights': 'mean_ipcw'})
 .assign(weighted_n = lambda x: x.n*x.mean_ipcw)
 .filter(['weighted_n'])
 .reset_index()
 .assign(labels = lambda x: x.labels.astype(str))
 .pivot(index='group', columns='labels', values='weighted_n')
 .assign(ascvd_10yr_weighted = lambda x: x['1']/(x['0']+x['1']))
 #.filter('ascvd_10yr_weighted')
 .reset_index()
 .rename_axis('', axis='index')
 .filter(['group','ascvd_10yr_weighted'])
)

labels,group,ascvd_10yr_weighted
,,
0,1,0.056921
1,2,0.059490
2,3,0.081543
3,4,0.103570


In [107]:
(preds
 .filter(['phase', 'person_id', 'labels', 'weights', 'group'])
 .groupby(['group', 'labels'])
 .agg({'person_id': 'count', 'weights': 'mean'})
 .rename(columns={'person_id': 'n', 'weights': 'mean_ipcw'})
 .assign(weighted_n = lambda x: x.n*x.mean_ipcw)
 .filter(['weighted_n'])
.reset_index()
 .assign(labels = lambda x: x.labels.astype(str))
 .pivot(index='group', columns='labels', values='weighted_n')
 .assign(ascvd_10yr_weighted = lambda x: x['1']/(x['0']+x['1']))
 .round(3)
#  .reset_index()
#  .rename_axis('', axis='index')
  .filter(['group','ascvd_10yr_weighted'])
)

labels,ascvd_10yr_weighted
group,
1,0.057
2,0.059
3,0.082
4,0.104


In [109]:
a = (preds
 .filter(['phase', 'person_id', 'labels', 'weights',])
 .groupby(['labels'])
 .agg({'person_id': 'count', 'weights': 'mean'})
 .rename(columns={'person_id': 'n', 'weights': 'mean_ipcw'})
 .assign(weighted_n = lambda x: x.n*x.mean_ipcw)
 .filter(['weighted_n'])
 .reset_index()
 .assign(labels = lambda x: x.labels.astype(str))
 .pivot(columns='labels', values='weighted_n')
 .sum(axis=0)
 #.assign(ascvd_10yr_weighted = lambda x: x['1']/(x['0']+x['1']))
 #.reset_index()
 #.rename_axis('', axis='index')
 #.filter(['ascvd_10yr_weighted'])
)
overall_ascvd_10yr_weighted = a['1']/(a['1']+a['0'])
overall_ascvd_10yr_weighted.round(3)

0.075